In [1]:
import numpy as np
import numpy.linalg as LA
from numpy.polynomial import Chebyshev as T
from scipy import optimize
from maxvolpy.maxvol import rect_maxvol, maxvol
import matplotlib.pyplot as plt
from gen_mat import GenMat
import gen_mat as gen
import grad_log_det as log_det
from numba import jit
from pyDOE import *
from sobol_lib import *
%matplotlib inline

In [2]:
def GenSobol(N = 200, dim = 2, seed = 2, rng=(-1.0, 1.0)):
    res = np.zeros((N, dim), dtype=float)
    rng_d = rng[1] - rng[0]
    for i in range(N):
        res[i, :], seed = i4_sobol ( dim, seed )
        res[i, :] = res[i, :]*rng_d + rng[0]

    return res

def C(x):
    M   = (x[:,0]-0.1)*30 + 30#45
    S   = (x[:,1]-0.1)*0.015 + 0.005#0.0125#x[:,1]
    V_0 = (x[:,2]-0.1)*0.008 + 0.002#0.006#x[:,2]
    k   = (x[:,3]-0.1)*4000 + 1000#3000
    P_0 = (x[:,4]-0.1)*20000+90000#100000#(x[:,4]-0.1)#100000#x[:,4]
    T_a = 293#(x[:,5]-0.1)*6+290#293#x[:,5]
    T_0 = 350#(x[:,6]-0.1)*20+340#350#x[:,6]

    A = P_0*S + 19.62*M - k*V_0/S
    V = S/(2*k)*(np.sqrt(A**2 + 4*k*P_0*V_0*T_a/T_0) - A)
    C = 2*np.pi*np.sqrt(M/(k + S**2*P_0*V_0*T_a/(T_0*V**2)))
    return C

In [3]:
np.max(gen.indeces_K_cut(5, 350))

5

In [4]:
np.random.seed(2)
dim = 5
log_det.dim = dim 

loss_f = lambda x: log_det.loss_func(x, poly=gen.cheb, ToGenDiff=False)
loss_grad = lambda x: log_det.grad(x, poly=gen.cheb)

N_COL = [350,400]
NUM_MAXVOL = 10000

#BIG matrix
M = 0.1+np.random.rand(NUM_MAXVOL, 1)
S = 0.1+np.random.rand(NUM_MAXVOL, 1)
V_0 = 0.1+np.random.rand(NUM_MAXVOL, 1)
k = 0.1+np.random.rand(NUM_MAXVOL, 1)
P_0 = 0.1+np.random.rand(NUM_MAXVOL, 1)
#T_a = 0.1+np.random.rand(NUM_MAXVOL, 1)
#T_0 = 0.1+np.random.rand(NUM_MAXVOL, 1)

X_BIG = np.hstack((M,S,V_0,k,P_0))#,T_a))#,T_0))

file = open('piston_sim_full_func_sobol.txt', 'w')
for l in range(len(N_COL)):
    for ind in range(0,4):
        num_col = N_COL[l]
        log_det.num_col = num_col
        num_of_points = N_COL[l] + 3*ind
        
        %time res = GenSobol(N = num_of_points, dim = dim, rng=(0, 1)) + 0.1
        #SMALL matrix
#         M = 0.1+np.random.rand(num_of_points, 1)
#         S = 0.1+np.random.rand(num_of_points, 1)
#         V_0 = 0.1+np.random.rand(num_of_points, 1)
#         k = 0.1+np.random.rand(num_of_points, 1)
#         P_0 = 0.1+np.random.rand(num_of_points, 1)
#         #T_a = 0.1+np.random.rand(num_of_points, 1)
#         #T_0 = 0.1+np.random.rand(num_of_points, 1)

#         X_SMALL = np.hstack((M,S,V_0,k,P_0))#,T_a))#,T_0))

#         x_0 = X_SMALL.ravel('F')

#         bnds_M = list((0.1, 1.1) for i in range(num_of_points))
#         bnds_S = list((0.1, 1.1) for i in range(num_of_points))
#         bnds_V_0 = list((0.1, 1.1) for i in range(num_of_points))
#         bnds_k = list((0.1, 1.1) for i in range(num_of_points))
#         bnds_P_0 = list((0.1, 1.1) for i in range(num_of_points))
#         #bnds_T_a = list((0.1, 1.1) for i in range(num_of_points))
#         #bnds_T_0 = list((0.1, 1.1) for i in range(num_of_points))
#         bnds = np.vstack((bnds_M,bnds_S,bnds_V_0, bnds_k, bnds_P_0))#, bnds_T_a))#, bnds_T_0))

#         #GD
#         res = np.inf
#         %time res = optimize.fmin_l_bfgs_b(loss_f, x_0, fprime = loss_grad, factr = 10.0, bounds = bnds)
#         #optimize.fmin_l_bfgs_b(lambda x: log_det.loss_func(x), x_0, fprime = log_det.grad, factr = 1.0, bounds = bnds)
        
#         #accuracy
#         M_gd = GenMat(num_col, res[0].reshape(-1, dim, order="F"), poly = gen.cheb, ToGenDiff=False)
#         c_gd = LA.solve(np.dot(M_gd.T, M_gd), np.dot(M_gd.T, C(res[0].reshape(-1, dim, order="F"))))
        M_maxvol = gen.GenMat(num_col, X_BIG, poly = gen.cheb, ToGenDiff=False)

        M_gd = GenMat(num_col, res, poly = gen.cheb, ToGenDiff=False)
        c_gd = LA.solve(np.dot(M_gd.T, M_gd), np.dot(M_gd.T, C(res)))

        file.write("Num of points = " + str(num_of_points) + "\n" +  "Num of columns = " + str(num_col) + "\n")
        print("Error_GD = ", LA.norm(C(X_BIG) - np.dot(M_maxvol, c_gd), np.inf) / LA.norm(C(X_BIG), np.inf), "\n")
        file.write("Error_GD = ")
        file.write(str(LA.norm(C(X_BIG) - np.dot(M_maxvol, c_gd), np.inf) / LA.norm(C(X_BIG), np.inf)) + "\n")
        
        #MAXVOL
        #row_indx,_ = rect_maxvol(M_maxvol, tol = 1.01, minK = num_of_points, maxK = num_of_points)

        #accuracy
        #c_maxvol = LA.solve(np.dot(M_maxvol[row_indx].T, M_maxvol[row_indx]), np.dot(M_maxvol[row_indx].T, C(X_BIG[row_indx])))

        #print("Error_MAXVOL = ", LA.norm(C(X_BIG) - np.dot(M_maxvol, c_maxvol), np.inf) / LA.norm(C(X_BIG), np.inf), "\n\n\n")

        #file.write("Error_MAXVOL = ")
        #file.write(str(LA.norm(C(X_BIG) - np.dot(M_maxvol, c_maxvol), np.inf) / LA.norm(C(X_BIG), np.inf)))
        file.write("\n\n\n")

        file.flush()
        
file.close()

CPU times: user 6.39 ms, sys: 758 µs, total: 7.14 ms
Wall time: 6.42 ms
Error_GD =  1.86071672947 

CPU times: user 21.8 ms, sys: 110 µs, total: 21.9 ms
Wall time: 5.49 ms
Error_GD =  0.734473156272 

CPU times: user 21.8 ms, sys: 108 µs, total: 21.9 ms
Wall time: 5.49 ms
Error_GD =  0.384057125536 

CPU times: user 22.1 ms, sys: 91 µs, total: 22.2 ms
Wall time: 5.54 ms
Error_GD =  0.244048830952 

CPU times: user 24.6 ms, sys: 108 µs, total: 24.7 ms
Wall time: 6.17 ms
Error_GD =  1.50367993875 

CPU times: user 24.6 ms, sys: 107 µs, total: 24.7 ms
Wall time: 6.17 ms
Error_GD =  0.713124178555 

CPU times: user 24.8 ms, sys: 112 µs, total: 24.9 ms
Wall time: 6.23 ms
Error_GD =  0.431157544486 

CPU times: user 25 ms, sys: 112 µs, total: 25.1 ms
Wall time: 6.29 ms
Error_GD =  0.474950826913 

